In [ ]:
# Applying the Methodology mentioned in the documentation to estimate 2025 population for each age cohort


In [ ]:
#STEP 0: Imports & Parameters

import pandas as pd

SURVIVAL_FACTOR = 0.95


In [ ]:
#Load & Inspect Input Data

census = pd.read_csv("/content/census_2011_aggregated.csv")
growth = pd.read_csv("/content/statewise_growthrate.csv")



In [ ]:
# Standardise column names
census.columns = census.columns.str.strip().str.lower()
growth.columns = growth.columns.str.strip().str.lower()

# Rename for consistency
growth = growth.rename(columns={
    'country/states/ut': 'state',
    'population growth rate': 'growthrate'
})

print("Census columns:", census.columns.tolist())
print("Growth columns:", growth.columns.tolist())

display(census.head())
display(growth.head())

Census columns: ['state', 'age_0_5', 'age_5_17', 'age_18_greater', 'age_not_stated']
Growth columns: ['state', 'growthrate']


state   age_0_5  age_5_17  age_18_greater  \
0  Andaman & Nicobar Islands     34721     77588          267802   
1             Andhra Pradesh   7725898  18759505        57295958   
2          Arunachal Pradesh    177268    412288          792169   
3                      Assam   3937549   8140172        19097809   
4                      Bihar  15837132  31665933        56154242   

   age_not_stated  
0             470  
1          799416  
2            2002  
3           30046  
4          442145

state  growthrate
0  Andaman & Nicobar Islands         0.2
1             Andhra Pradesh         3.5
2          Arunachal Pradesh         9.1
3                      Assam         9.3
4                      Bihar        14.4

In [ ]:
#STEP 1: Prepare 2011 Total & Child Population

# Total population 2011 (including age_not_stated)
census['total_pop_2011'] = (
    census['age_0_5'] +
    census['age_5_17'] +
    census['age_18_greater'] +
    census['age_not_stated']
)

# Child population 2011
census['child_pop_2011'] = census['age_0_5'] + census['age_5_17']

# Child cohort ratios
census['ratio_0_5'] = census['age_0_5'] / census['child_pop_2011']
census['ratio_5_17'] = census['age_5_17'] / census['child_pop_2011']

display(
    census[['state', 'total_pop_2011', 'ratio_0_5', 'ratio_5_17']].head()
)

print("Ratio check (should be ~1):")
display(
    census.assign(
        ratio_sum=census['ratio_0_5'] + census['ratio_5_17']
    )[['state', 'ratio_sum']].head()
)


state  total_pop_2011  ratio_0_5  ratio_5_17
0  Andaman & Nicobar Islands          380581   0.309156    0.690844
1             Andhra Pradesh        84580777   0.291704    0.708296
2          Arunachal Pradesh         1383727   0.300681    0.699319
3                      Assam        31205576   0.326018    0.673982
4                      Bihar       104099452   0.333392    0.666608

Ratio check (should be ~1):


state  ratio_sum
0  Andaman & Nicobar Islands        1.0
1             Andhra Pradesh        1.0
2          Arunachal Pradesh        1.0
3                      Assam        1.0
4                      Bihar        1.0

In [ ]:
#STEP 2: Estimate Total Population (2025)

df['total_pop_2025'] = df['total_pop_2011'] * (1 + df['growthrate'] / 100)

print("2025 population estimate sample:")
display(
    df[['state', 'total_pop_2011', 'growthrate', 'total_pop_2025']].head()
)

print("India-level estimated population 2025:")
print(df['total_pop_2025'].sum())


2025 population estimate sample:


state  total_pop_2011  growthrate  total_pop_2025
0  Andaman & Nicobar Islands          380581         0.2    3.813422e+05
1             Andhra Pradesh        84580777         3.5    8.754110e+07
2          Arunachal Pradesh         1383727         9.1    1.509646e+06
3                      Assam        31205576         9.3    3.410769e+07
4                      Bihar       104099452        14.4    1.190898e+08

India-level estimated population 2025:
1316570450.061


In [ ]:
#STEP 3: Adult & Child Population (2025)

df['adult_18_plus_2025'] = df['total_pop_2011'] * SURVIVAL_FACTOR
df['child_0_17_2025'] = df['total_pop_2025'] - df['adult_18_plus_2025']

display(
    df[['state', 'adult_18_plus_2025', 'child_0_17_2025']].head()
)




state  adult_18_plus_2025  child_0_17_2025
0  Andaman & Nicobar Islands           361551.95     1.979021e+04
1             Andhra Pradesh         80351738.15     7.189366e+06
2          Arunachal Pradesh          1314540.65     1.951055e+05
3                      Assam         29645297.20     4.462397e+06
4                      Bihar         98894479.40     2.019529e+07

In [ ]:
# STEP 4: Split Child Population into Cohorts (2025)

df['age_0_5_2025'] = df['child_0_17_2025'] * df['ratio_0_5']
df['age_5_17_2025'] = df['child_0_17_2025'] * df['ratio_5_17']

display(
    df[['state', 'age_0_5_2025', 'age_5_17_2025', 'adult_18_plus_2025']].head()
)

print("Cohort sum check:")
display(
    df.assign(
        cohort_sum=
        df['age_0_5_2025'] +
        df['age_5_17_2025'] +
        df['adult_18_plus_2025']
    )[['state', 'cohort_sum', 'total_pop_2025']].head()
)


state  age_0_5_2025  age_5_17_2025  adult_18_plus_2025
0  Andaman & Nicobar Islands  6.118263e+03   1.367195e+04           361551.95
1             Andhra Pradesh  2.097167e+06   5.092199e+06         80351738.15
2          Arunachal Pradesh  5.866442e+04   1.364411e+05          1314540.65
3                      Assam  1.454820e+06   3.007578e+06         29645297.20
4                      Bihar  6.732945e+06   1.346235e+07         98894479.40

Cohort sum check:


state    cohort_sum  total_pop_2025
0  Andaman & Nicobar Islands  3.813422e+05    3.813422e+05
1             Andhra Pradesh  8.754110e+07    8.754110e+07
2          Arunachal Pradesh  1.509646e+06    1.509646e+06
3                      Assam  3.410769e+07    3.410769e+07
4                      Bihar  1.190898e+08    1.190898e+08

In [ ]:
# STEP  5: Final Output

final = df[
    [
        'state',
        'age_0_5_2025',
        'age_5_17_2025',
        'adult_18_plus_2025',
        'total_pop_2025'
    ]
]



print(" 2025 population estimation:")
display(final.head())


 2025 population estimation:


state  age_0_5_2025  age_5_17_2025  adult_18_plus_2025  \
0  Andaman & Nicobar Islands  6.118263e+03   1.367195e+04           361551.95   
1             Andhra Pradesh  2.097167e+06   5.092199e+06         80351738.15   
2          Arunachal Pradesh  5.866442e+04   1.364411e+05          1314540.65   
3                      Assam  1.454820e+06   3.007578e+06         29645297.20   
4                      Bihar  6.732945e+06   1.346235e+07         98894479.40   

   total_pop_2025  
0    3.813422e+05  
1    8.754110e+07  
2    1.509646e+06  
3    3.410769e+07  
4    1.190898e+08

In [ ]:
final.head(35)

state  age_0_5_2025  age_5_17_2025  \
0   Andaman & Nicobar Islands  6.118263e+03   1.367195e+04   
1              Andhra Pradesh  2.097167e+06   5.092199e+06   
2           Arunachal Pradesh  5.866442e+04   1.364411e+05   
3                       Assam  1.454820e+06   3.007578e+06   
4                       Bihar  6.732945e+06   1.346235e+07   
5                  Chandigarh  1.940808e+04   4.286347e+04   
6                Chhattisgarh  1.326181e+06   2.837686e+06   
7        Dadra & Nagar Haveli  8.452615e+03   1.560702e+04   
8                 Daman & Diu  9.760005e+03   1.845665e+04   
9                       Delhi  1.203852e+06   2.707738e+06   
10                        Goa  2.589293e+04   5.432705e+04   
11                    Gujarat  3.264535e+06   7.010213e+06   
12                    Haryana  1.362202e+06   2.947546e+06   
13           Himachal Pradesh  2.112084e+05   4.752518e+05   
14            Jammu & Kashmir  5.261103e+05   1.003929e+06   
15                  Jharkhand  1.882753e+06   3.890170e+06   
16                  Karnataka  2.169362e+06   4.673312e+06   
17                     Kerala  9.480698e+05   2.058476e+06   
18                Lakshadweep  1.002197e+03   2.221453e+03   
19             Madhya Pradesh  3.878588e+06   8.322716e+06   
20                Maharashtra  4.494468e+06   9.777073e+06   
21                    Manipur  1.230217e+05   2.796452e+05   
22                  Meghalaya  1.477192e+05   2.706121e+05   
23                    Mizoram  5.315252e+04   1.015535e+05   
24                   Nagaland  8.260361e+04   1.963652e+05   
25                     Odisha  1.474660e+06   3.268426e+06   
26                Pondicherry  2.794192e+04   6.066274e+04   
27                     Punjab  9.453375e+05   2.217403e+06   
28                  Rajasthan  3.445820e+06   7.316285e+06   
29                     Sikkim  2.212430e+04   6.396706e+04   
30                 Tamil Nadu  1.776128e+06   3.995634e+06   
31                    Tripura  1.652710e+05   3.527513e+05   
32              Uttar Pradesh  9.065757e+06   2.110591e+07   
33                Uttarakhand  4.580705e+05   1.064960e+06   
34                West Bengal  2.687433e+06   6.257626e+06   

    adult_18_plus_2025  total_pop_2025  
0         3.615520e+05    3.813422e+05  
1         8.035174e+07    8.754110e+07  
2         1.314541e+06    1.509646e+06  
3         2.964530e+07    3.410769e+07  
4         9.889448e+07    1.190898e+08  
5         1.002678e+06    1.064949e+06  
6         2.426794e+07    2.843181e+07  
7         3.265235e+05    3.505832e+05  
8         2.310846e+05    2.593013e+05  
9         1.594854e+07    1.986013e+07  
10        1.385618e+06    1.465838e+06  
11        5.741771e+07    6.769246e+07  
12        2.408389e+07    2.839364e+07  
13        6.521372e+06    7.207832e+06  
14        1.191424e+07    1.344428e+07  
15        3.133873e+07    3.711165e+07  
16        5.804053e+07    6.488321e+07  
17        3.173576e+07    3.474230e+07  
18        6.124935e+04    6.447300e+04  
19        6.899547e+07    8.119677e+07  
20        1.067556e+08    1.210272e+08  
21        2.713004e+06    3.115671e+06  
22        2.818545e+06    3.236876e+06  
23        1.042346e+06    1.197052e+06  
24        1.879577e+06    2.158546e+06  
25        3.987551e+07    4.461859e+07  
26        1.185555e+06    1.274160e+06  
27        2.635617e+07    2.951891e+07  
28        6.512102e+07    7.588312e+07  
29        5.800482e+05    6.661395e+05  
30        6.853968e+07    7.431144e+07  
31        3.490221e+06    4.008243e+06  
32        1.898217e+08    2.199934e+08  
33        9.581977e+06    1.110501e+07  
34        8.671231e+07    9.565737e+07

In [ ]:
# Rounding off

# -----------------------------
# ROUND ALL POPULATION COLUMNS
# -----------------------------
population_cols = [
    'age_0_5_2025',
    'age_5_17_2025',
    'adult_18_plus_2025',
    'total_pop_2025'
]

final[population_cols] = final[population_cols].round().astype(int)

display(final.head(35))


/tmp/ipython-input-3016774853.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[population_cols] = final[population_cols].round().astype(int)


state  age_0_5_2025  age_5_17_2025  \
0   Andaman & Nicobar Islands          6118          13672   
1              Andhra Pradesh       2097167        5092199   
2           Arunachal Pradesh         58664         136441   
3                       Assam       1454820        3007578   
4                       Bihar       6732945       13462349   
5                  Chandigarh         19408          42863   
6                Chhattisgarh       1326181        2837686   
7        Dadra & Nagar Haveli          8453          15607   
8                 Daman & Diu          9760          18457   
9                       Delhi       1203852        2707738   
10                        Goa         25893          54327   
11                    Gujarat       3264535        7010213   
12                    Haryana       1362202        2947546   
13           Himachal Pradesh        211208         475252   
14            Jammu & Kashmir        526110        1003929   
15                  Jharkhand       1882753        3890170   
16                  Karnataka       2169362        4673312   
17                     Kerala        948070        2058476   
18                Lakshadweep          1002           2221   
19             Madhya Pradesh       3878588        8322716   
20                Maharashtra       4494468        9777073   
21                    Manipur        123022         279645   
22                  Meghalaya        147719         270612   
23                    Mizoram         53153         101554   
24                   Nagaland         82604         196365   
25                     Odisha       1474660        3268426   
26                Pondicherry         27942          60663   
27                     Punjab        945338        2217403   
28                  Rajasthan       3445820        7316285   
29                     Sikkim         22124          63967   
30                 Tamil Nadu       1776128        3995634   
31                    Tripura        165271         352751   
32              Uttar Pradesh       9065757       21105907   
33                Uttarakhand        458071        1064960   
34                West Bengal       2687433        6257626   

    adult_18_plus_2025  total_pop_2025  
0               361552          381342  
1             80351738        87541104  
2              1314541         1509646  
3             29645297        34107695  
4             98894479       119089773  
5              1002678         1064949  
6             24267938        28431805  
7               326524          350583  
8               231085          259301  
9             15948544        19860134  
10             1385618         1465838  
11            57417707        67692455  
12            24083889        28393637  
13             6521372         7207832  
14            11914237        13444276  
15            31338727        37111651  
16            58040532        64883205  
17            31735758        34742303  
18               61249           64473  
19            68995469        81196772  
20           106755616       121027157  
21             2713004         3115671  
22             2818545         3236876  
23             1042346         1197052  
24             1879577         2158546  
25            39875507        44618594  
26             1185555         1274160  
27            26356171        29518912  
28            65121015        75883120  
29              580048          666140  
30            68539678        74311441  
31             3490221         4008243  
32           189821724       219993387  
33             9581977        11105007  
34            86712309        95657369

In [ ]:
final.to_csv("population_estimation_2025.csv", index=False)

In [ ]:
from google.colab import files

files.download('population_estimation_2025.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>